# 2 Player FPSB Auction with symmetric unfirorm valuation distributions

## Imports

In [1]:
import os
import sys
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)

In [2]:
import torch
import torch.nn as nn
import torch.nn.utils as ut
from torch.optim.optimizer import Optimizer, required

In [3]:
from bnelearn.strategy import NeuralNetStrategy, TruthfulStrategy
from bnelearn.bidder import Bidder
from bnelearn.mechanism import FirstPriceSealedBidAuction, VickreyAuction
from bnelearn.optimizer import ES
from bnelearn.environment import Environment

In [4]:
from tensorboardX import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt

In [5]:
torch.cuda.is_available()

True

## Settings

In [6]:
## Experiment setup
n_players = 2
n_items = 1
# valuation distribution
u_lo =0
u_hi =10

## Environment settings
#training batch size
batch_size = 2**16
input_length = 1

# model architecture
size_hidden_layer = 10

# optimization params
epoch = 9000
learning_rate = 5e-3
sigma = .1 #ES noise parameter
n_perturbations = 64

# plot and log training options
plot_epoch = 50
plot_points = min(150, batch_size)
sample_points = torch.from_numpy(np.linspace(u_lo, u_hi, u_hi+1)).float().view(-1, n_items).cuda()

# tensorboard writer settings

## Setting up the Environment

In [7]:
# for evaluation
def optimal_bid(valuation):
    return valuation / 2

In [8]:
# Wrapper transforming a strategy to bidder, used by the optimizer
def strat_to_bidder(strategy, batch_size):
    return Bidder.uniform(u_lo,u_hi, strategy, batch_size = batch_size, n_players=2)

In [9]:
model = NeuralNetStrategy(input_length, size_hidden_layer = size_hidden_layer, requires_grad=False).cuda()

In [10]:
mechanism = FirstPriceSealedBidAuction(cuda = True)
env = Environment(mechanism,
                  environment_agents = [], #dynamically built
                  max_env_size = 1, #
                  batch_size = batch_size,
                  n_players =2,
                  strategy_to_bidder_closure = strat_to_bidder
                 )
optimizer = ES(model=model, environment = env,
               lr = learning_rate, sigma=sigma, n_perturbations=n_perturbations)

In [11]:
def log_hyperparams(writer):
    writer.add_scalar('hyperparams/batch_size', batch_size)
    writer.add_scalar('hyperparams/size_hidden_layer', size_hidden_layer)
    writer.add_scalar('hyperparams/learning_rate', learning_rate)
    writer.add_scalar('hyperparams/sigma', sigma)
    writer.add_scalar('hyperparams/n_perturbations', n_perturbations)
    
    

## Training

In [ ]:
torch.cuda.empty_cache()
writer = SummaryWriter()
log_hyperparams(writer)
for e in range(epoch+1):
    if e % plot_epoch == 0:
        # plot current function output
        bidder = strat_to_bidder(model, batch_size)
        bidder.draw_valuations_()
        v = bidder.valuations
        b = bidder.get_action()
        share = (b/v).mean()
        utility = -optimizer.step()
        writer.add_scalar('eval/utility', utility, e)
        writer.add_scalar('eval/share', share, e)
        
        if e==0:
            writer.add_graph(model, bidder.valuations)
            
        print("Epoch {}: \tavg bid: {:2f}, \tutility: {:2f}".format(e, share, utility))
                
        
        # subsample points and plot
        v = v.detach().cpu().numpy()[:plot_points]
        b= b.detach().cpu().numpy()[:plot_points]
        optimal = optimal_bid(v)
        
        fig = plt.figure()
        plt.plot(v,b, 'o', v, optimal, 'r-')
        
        writer.add_figure('eval/bid_function', fig, e)
        
        plt.show()
    else:
        utility = -optimizer.step()
        writer.add_scalar('eval/utility', utility, e)    
        
torch.cuda.empty_cache()
writer.close()

C:\Users\stefan\Anaconda3\envs\bnelearn\lib\site-packages\torch\nn\functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch 0: 	avg bid: 0.057419, 	utility: 0.000000
Epoch 50: 	avg bid: 0.427440, 	utility: 4.279208
Epoch 100: 	avg bid: 1.168555, 	utility: 3.712126
Epoch 150: 	avg bid: 1.010177, 	utility: 3.188411
Epoch 200: 	avg bid: 1.219319, 	utility: 2.723931
Epoch 250: 	avg bid: 1.403139, 	utility: 2.320334
Epoch 300: 	avg bid: 1.633250, 	utility: 1.945533
Epoch 350: 	avg bid: 1.597993, 	utility: 1.638396
Epoch 400: 	avg bid: 1.911089, 	utility: 1.379446
Epoch 450: 	avg bid: 2.174857, 	utility: 1.143945
Epoch 500: 	avg bid: 2.089782, 	utility: 0.961506
Epoch 550: 	avg bid: 2.814307, 	utility: 0.771051
Epoch 600: 	avg bid: 2.131930, 	utility: 0.625491
Epoch 650: 	avg bid: 3.263984, 	utility: 0.500088
Epoch 700: 	avg bid: 2.445767, 	utility: 0.404735
Epoch 750: 	avg bid: 5.620318, 	utility: 0.315856
Epoch 800: 	avg bid: 2.279997, 	utility: 0.250386
Epoch 850: 	avg bid: 2.745893, 	utility: 0.175985
Epoch 900: 	avg bid: 2.419563, 	utility: 0.101535
Epoch 950: 	avg bid: 2.488320, 	utility: 0.054606
Epo